<a href="https://colab.research.google.com/github/isdeniz/msc-2024/blob/main/llama3_msc_summary_and_preds_with_70b_and_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!python3.10 -m pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
!python3.10 -m pip install langchain-core

In [4]:
!python3.10 -m pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3", num_ctx=8192, temperature=0)

In [6]:
pip install tqdm

In [7]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [8]:
test_sample = pd.read_pickle('data/msc_test_sample_summarized_llama3_70b_preds.pkl')
print('Number of issues: ', test_sample.shape)

Number of issues:  (28, 30)


In [9]:
test_sample.head()

,index,original_issue_title,preprocessed_title,original_issue_body,preprocessed_body,original_label,text,issue_label,labels,roberta_prediction,...,llama3_70b_classification_corrected,llama3_70b_confidence,llama3_70b_confidence_corrected,llama3_70b_understandability,llama3_70b_understandability_corrected,round2_70b,llama3_70b_body_summary,llama3_70b_body_summary_check,llama3_70b_body_summary_corrected,llama3_70b_prediction_round2
68,3007,Jpa namedQuery for different data types error,jpa namedquery for different data types error,``` java\n@Entity\n@Data\n@Cacheable\n@Table(n...,java entity data cacheable table name user pub...,['for: stackoverflow'],time 2016-01-15T09:05:31Z author NONE repo spr...,question,2,question,...,,,,,,True,This is a list of Java libraries and their cor...,OK,This is a list of Java libraries and their cor...,Classification: other\nPrediction Confidence: ...
85,9537,Druid query returns 500 error with javax.servl...,druid query returns 500 error with javax.servl...,### Affected Version\r\n0.22.1\r\n\r\n### Desc...,affected version 0.22.1 description after upgr...,"['Bug', 'Area - SQL']",time 2022-09-15T07:08:19Z author CONTRIBUTOR r...,bug,0,bug,...,,,,,,True,Here is a summary of the error:\n\n* The error...,OK,Here is a summary of the error:\n\n* The error...,Classification: bug\nPrediction Confidence: 95...
132,3755,WLS 12c spring-boot war deploy fail,wls 12c spring boot war deploy fail,>>The ResourceConfig instance does not contain...,the resourceconfig instance does not contain a...,['for: stackoverflow'],time 2017-02-23T09:28:08Z author NONE repo spr...,question,2,question,...,,,,,,True,"This is a Java application log output, specifi...",OK,"This is a Java application log output, specifi...",Classification: bug\nPrediction Confidence: 90...
241,3767,My Game Crashes,my game crashes,**Platform**: *Windows*\r\n\r\n**Build**: *The...,platform windows build the build number 104.10...,['bug'],time 2020-06-24T11:29:22Z author NONE repo Anu...,bug,0,bug,...,,,,,,True,"This appears to be a Java crash log, which pro...",OK,"This appears to be a Java crash log, which pro...",Classification: bug\nPrediction Confidence: 90...
893,10196,[Bug] shadow topic replicator may fail due to ...,bug shadow topic replicator may fail due to re...,### Search before asking\r\n\r\n- [X] I search...,search before asking x i searched in the issue...,['type/bug'],time 2022-12-04T10:18:10Z author CONTRIBUTOR r...,bug,0,bug,...,,,,,,True,This is a log output from a test case in Apach...,last two sentences are removed,This is a log output from a test case in Apach...,Classification: bug\nPrediction Confidence: 10...


In [10]:
with tqdm(total=test_sample.shape[0], ncols=100) as pbar:
    for idx in test_sample.index:
        try:
            test_sample.loc[idx, 'llama3_8b_prediction_round2'] = llm.invoke(f"""Classify the following issue as "bug", "enhancement", "question" or "other" and rate your classification confidence and issue understandability in range 0 - 100. Your answer should only include:
Classification: your classification answer
Prediction Confidence: your level of confidence
Issue Understandability: level of understandability
Do not provide any explanation.
Issue Title: {test_sample.loc[idx,'original_issue_title']}
Issue Body: {test_sample.loc[idx,'llama3_70b_body_summary_corrected']}
""")
            pbar.update(1)
        except Exception as e:
            print(f'Exception at index {idx}: {e}')
            continue

print(f'Categorization of {test_sample.shape[0]} issues is complete')

100%|███████████████████████████████████████████████████████████████| 28/28 [00:14<00:00,  1.87it/s]

Categorization of 28 issues is complete


In [11]:
test_sample['llama3_8b_prediction_round2'].value_counts()

,count
llama3_8b_prediction_round2,
Classification: bug\nPrediction Confidence: 80\nIssue Understandability: 90,11
Classification: Bug\nPrediction Confidence: 80\nIssue Understandability: 90,4
Classification: Bug\nPrediction Confidence: 90\nIssue Understandability: 80,4
Classification: enhancement\nPrediction Confidence: 80\nIssue Understandability: 90,4
Classification: bug\nPrediction Confidence: 90\nIssue Understandability: 80,2
Classification: question\nPrediction Confidence: 80\nIssue Understandability: 90,1
Classification: bug\nPrediction Confidence: 80\nIssue Understandability: 60,1
Classification: question\nPrediction Confidence: 100\nIssue Understandability: 0,1


In [12]:
test_sample.to_pickle('msc_test_sample_summarized_llama3_70b_and_8b_preds.pkl')